In [1]:
!pip3 install pandas numpy torch tqdm 

In [2]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [3]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [4]:
tokenizer = BpeTokenizer()

In [5]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [02:50<00:00, 10.51it/s]  


# Создаем датасеты и Collator

In [6]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [00:03<00:00, 296.10it/s]


# Создаем модель

In [7]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [8]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=2, dropout=0.1)

# Создаем Trainer и запускаем обучение

In [14]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=8,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=1e-2,
    ignore_index=tokenizer.pad_token_id
)

In [17]:
trainer.train()

x shape: torch.Size([32, 127]), y shape: torch.Size([32, 127])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

# Оцениваем качество и проверяем жадную и случайную генерацию

In [ ]:
trainer.evaluate()

In [ ]:
generate(model, tokenizer, temperature=0)

In [ ]:
generate(model, tokenizer, temperature=0.5, top_k=20)